# 자연어 처리 크롤링 파일(멜론)
- 예외적으로 가상환경 My_38
- 2000년부터 크롤링 막혀서 에러 발생 
- 가사 정보 없음 / 미성년자 접근 불가 : try except 구문 사용

In [23]:
'''
멜론에서 시대별 차트
-> 맨 아래 top100
1번 -> 2020~2023
2번 -> 2010~2019
3번 -> 2000~2009
4번 -> 1990~1999
'''
start_year, max_year = 2010, 2014

import os
import sys
import urllib.request
import json
import datetime
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import time
import re
from selenium.webdriver.common.by import By # find_element 때문에
from selenium.webdriver.common.keys import Keys


cnt = 1
# max_year=2023 # 몇번째 페이지까지 넘길건지

data_dict = {"가수":[],
             "곡명":[],
             "가사":[],
             "장르":[],
             "발매일":[]
             }

# main문
driver = webdriver.Chrome()

for page_num in range(start_year, max_year+1):
    # 멜론 주소 page_num = 연도가 들어감 
    url = f"https://www.melon.com/chart/age/index.htm?chartType=YE&chartGenre=KPOP&chartDate={page_num}"

    # URL에서 HTML 가져오기
    driver.get(url)

    # BeautifulSoup을 사용하여 Selenium으로 열린 웹 페이지의 HTML 파싱
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 파싱된 HTML 출력
    # print(soup.prettify())
    # time.sleep(2)

    lst50 = soup.select("#lst50")
    lst100 = soup.select("#lst100")
    print(len(lst50), len(lst100))

    lst = lst50 + lst100

    for row in lst[:]:
        link_info = row.select("a")[1]
        print("="*50)
        # print(link_info.attrs["onclick"])

        num = link_info.attrs["onclick"].split("'")[-2]

        song_page = f"https://www.melon.com/song/detail.htm?songId={num}"
        print(num)
        print()
        driver.get(song_page)
        #######################################
        try : 
            artist = driver.find_elements(By.CLASS_NAME, 'artist')
            song_name = driver.find_elements(By.CLASS_NAME, 'song_name')
            lyric = driver.find_elements(By.CLASS_NAME, 'lyric')
            meta_data = driver.find_elements(By.CLASS_NAME, 'list')[0].text.split("\n")
            driver.implicitly_wait(60)
    
            lyrics = lyric[0].text
    
            print(artist[0].text) # 가수
            print(song_name[0].text) # 곡명
            # print(lyrics) # 가사
            # print(meta_data[5]) # 장르
            # print(meta_data[3]) # 발매일
            
            data_dict["가수"].append(artist[0].text)
            data_dict["곡명"].append(song_name[0].text)
            data_dict["가사"].append(lyrics)
            data_dict["장르"].append(meta_data[5])
            data_dict["발매일"].append(meta_data[3])
        except:
            print(song_page)
            
        #######################################

        #print()
        time.sleep(1) # sleep 없으면 속도차로 데이터 씹힘 
        driver.back()  # 이전 창으로 복구

driver.quit()

50 50
2741416
미쓰에이
Bad Girl Good Girl
2705410
아이유
잔소리 (With 2AM 슬옹)
2733249
MC몽
죽을 만큼 아파서 (Feat. 멜로우)
2544949
포맨 (4MEN)
못해 (Feat. 美)
2549955
2am
죽어도 못 보내
2554001
소녀시대 (GIRLS' GENERATION)
Oh!
2684572
원더걸스
2 Different Tears
2587788
티아라
너 때문에 미쳐
2776748
Homme (창민, 이현)
밥만 잘 먹더라
2930968
2NE1
Go Away
2579459
카라
루팡 (Lupin)
2670748
다비치
시간아 멈춰라
2930967
2NE1
Can't Nobody
2540268
CNBLUE (씨엔블루)
외톨이야
2951700
Supreme Team, 영준
그땐 그땐 그땐
2546258
이승기
사랑이 술을 가르쳐 (Feat. 백찬 From 8eight)
2627301
뜨거운 감자
고백
2679548
CNBLUE (씨엔블루)
Love
2783154
https://www.melon.com/song/detail.htm?songId=2783154
2688949
4minute
HUH
2621364
애프터스쿨
뱅(Bang) !
2611648
소녀시대 (GIRLS' GENERATION)
Run Devil Run
2667293
f(x)
NU 예삐오 (NU ABO)
2629049
시크릿
Magic
2529931
현아
Change (Feat. 용준형 From Beast)
2948449
미쓰에이
Breathe
2967751
강승윤
본능적으로 (Feat. Swings)
2705414
Supreme Team
땡땡땡
2596161
MC몽, 서인영
Bubble Love
2609744
2am
잘못했어
2626610
비
널 붙잡을 노래
2511561
가인, 조권
우리 사랑하게 됐어요.
2682148
디셈버
별이될께
2979880
2am
전활 받지 않는 너에게
2987448
소녀시대 (GIRLS' GENERATIO

In [24]:
# print(data_dict)
driver.quit()
df = pd.DataFrame.from_dict(data=data_dict)
print(df)

df.to_csv(f'melon_chart{start_year}to{max_year}.csv', sep=',', na_rep='NaN', encoding='utf-8')

            가수                       곡명  \
0         미쓰에이       Bad Girl Good Girl   
1          아이유        잔소리 (With 2AM 슬옹)   
2          MC몽    죽을 만큼 아파서 (Feat. 멜로우)   
3    포맨 (4MEN)             못해 (Feat. 美)   
4          2am                 죽어도 못 보내   
..         ...                      ...   
388       EXID                      위아래   
389   어쿠스틱 콜라보                   묘해, 너와   
390        김범수                      끝사랑   
391   첸 (CHEN)                   최고의 행운   
392        박명수  명수네 떡볶이 (Feat. 김예림, UL)   

                                                    가사          장르         발매일  
0    You don't know me\n\nYou don't know me\n\nYou ...          댄스  2010.07.01  
1    늦게 다니지좀 마\n\n술은 멀리좀 해봐\n\n열살짜리 애처럼 말을 안듣니\n\n정...         발라드  2010.06.03  
2    I found the way to\nlet you leave\nI never rea...        랩/힙합  2010.06.24  
3    이제는 밥을 먹어도\n눈물 없이는 삼키지 못해\n억지로 먹고 먹어도 속이 늘 허전해...         발라드  2010.01.18  
4    어려도 아픈건 똑같아\n세상을 잘 모른다고\n아픈걸 모르진 않아\n괜찮아 질거라고 ...         발라드  2010.01.21

In [25]:
import pandas as pd

In [26]:
pd.read_csv(f'melon_chart{start_year}to{max_year}.csv')["가사"][0]

"You don't know me\n\nYou don't know me\n\nYou don't know me\n\nYou don't know me\n\nSo shut off boy\n\nSo shut off boy\n\nSo shut off boy\n\nSo shut off shut off\n\n앞에선 한마디도 못하더니\n\n뒤에선 내 얘길 안 좋게 해\n\n참 어이가 없어\n\nHello hello hello\n\n나 같은 여잔 처음\n\n으로 으로 으로\n\n본 것 같은데 왜 나를 판단하니\n\n내가 혹시 두려운 거니\n\n겉으론 Bad girl\n\n속으론 Good girl\n\n나를 잘 알지도 못하면서\n\n내 겉모습만 보면서\n\n한심한 여자로 보는\n\n너의 시선이 난 너무나 웃겨\n\n춤출 땐 Bad girl\n\n사랑은 Good girl\n\n춤추는 내 모습을 볼 때는\n\n넋을 놓고 보고서는\n\n끝나니 손가락질하는\n\n그 위선이 난 너무나 웃겨\n\n이런 옷 이런 머리 모양으로\n\n이런 춤을 추는 여자는 뻔해\n\n하 네가 더 뻔해\n\nHello hello hello\n\n자신 없으면 저\n\n뒤로 뒤로 뒤로\n\n물러서면 되지 왜 자꾸 떠드니\n\n네 속이 훤히 보이는 건 아니\n\n겉으론 Bad girl\n\n속으론 Good girl\n\n나를 잘 알지도 못하면서\n\n내 겉모습만 보면서\n\n한심한 여자로 보는\n\n너의 시선이 난 너무나 웃겨\n\n춤출 땐 Bad girl\n\n사랑은 Good girl\n\n춤추는 내 모습을 볼 때는\n\n넋을 놓고 보고서는\n\n끝나니 손가락질하는\n\n그 위선이 난 너무나 웃겨\n\n날 감당 할 수 있는 남잘 찾아요\n\n진짜 남자를 찾아요\n\n말로만 남자다운 척할 남자 말고\n\n날 불안해 하지 않을 남잔 없나요\n\n자신감이 넘쳐서\n\n내가 나일 수 있게\n\n자유롭게 두고 멀리서 바라보는\n\n겉으론 Bad girl\n\n속으론 Good girl\n\n나를 잘 알지도 못하면서\n\n내

In [27]:
'''
멜론에서 시대별 차트
-> 맨 아래 top100
1번 -> 2020~2023
2번 -> 2010~2019
3번 -> 2000~2009
4번 -> 1990~1999
'''
start_year, max_year = 2005, 2009



cnt = 1
# max_year=2023 # 몇번째 페이지까지 넘길건지

data_dict = {"가수":[],
             "곡명":[],
             "가사":[],
             "장르":[],
             "발매일":[]
             }

# main문
driver = webdriver.Chrome()

for page_num in range(start_year, max_year+1):
    url = f"https://www.melon.com/chart/age/index.htm?chartType=YE&chartGenre=KPOP&chartDate={page_num}"

    # URL에서 HTML 가져오기
    driver.get(url)

    # BeautifulSoup을 사용하여 Selenium으로 열린 웹 페이지의 HTML 파싱
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 파싱된 HTML 출력
    # print(soup.prettify())
    # time.sleep(2)

    lst50 = soup.select("#lst50")
    lst100 = soup.select("#lst100")
    print(len(lst50), len(lst100))

    lst = lst50 + lst100

    for row in lst[:]:
        link_info = row.select("a")[1]
        print("="*50)
        # print(link_info.attrs["onclick"])

        num = link_info.attrs["onclick"].split("'")[-2]

        song_page = f"https://www.melon.com/song/detail.htm?songId={num}"
        print(num)
        print()
        driver.get(song_page)
        #######################################
        try : 
            artist = driver.find_elements(By.CLASS_NAME, 'artist')
            song_name = driver.find_elements(By.CLASS_NAME, 'song_name')
            lyric = driver.find_elements(By.CLASS_NAME, 'lyric')
            meta_data = driver.find_elements(By.CLASS_NAME, 'list')[0].text.split("\n")
            driver.implicitly_wait(60)
    
            lyrics = lyric[0].text
    
            print(artist[0].text) # 가수
            print(song_name[0].text) # 곡명
            # print(lyrics) # 가사
            # print(meta_data[5]) # 장르
            # print(meta_data[3]) # 발매일
            
            data_dict["가수"].append(artist[0].text)
            data_dict["곡명"].append(song_name[0].text)
            data_dict["가사"].append(lyrics)
            data_dict["장르"].append(meta_data[5])
            data_dict["발매일"].append(meta_data[3])
        except:
            print(song_page)
            
        # print(len(meta_data)) # 1

        # cnt = 1
        # for i in meta_data[0].text.split("\n"):
        #     print(f"{cnt} => {i}")
        #     cnt += 1

        
        #######################################

        #print()
        time.sleep(1)
        driver.back()  # 이전 창으로 복구

driver.quit()


df = pd.DataFrame.from_dict(data=data_dict)
print(df)

df.to_csv(f'melon_chart{start_year}to{max_year}.csv', sep=',', na_rep='NaN', encoding='utf-8')

50 50
711627
SG 워너비
죄와벌
732512
버즈
겁쟁이
837567
윤도현
사랑했나봐
711626
SG 워너비
살다가
711628
SG 워너비
광(狂)
905857
엠투엠 (M To M)
세글자 (드라마 '루루공주' 김정은 메인 테마)
866705
김종국
제자리 걸음
732519
버즈
가시
845586
MC몽
I Love U Oh Thank U (Feat. 김태우 Of God)
631232
채연
둘이서 (CF - 빙그레 '바나나맛 우유')
864916
KCM
너에게 전하는 아홉가지 바램
864906
KCM
Smile Again
698972
테이
사랑은...하나다
847132
빅마마
체념 後(후)
665226
나얼
귀로
898497
견우 (Kyunwoo)
내 눈물이 하는 말
933372
에픽하이 (EPIK HIGH)
Fly (Feat. Amin. J of Soulciety)
732373
모세
사랑인걸
866710
김종국
사랑스러워
732516
버즈
나에게로 떠나는 여행
866714
김종국
별, 바람, 햇살 그리고 사랑
672010
BMK
꽃피는 봄이오면
742311
별
안부 (Duet With 나윤권)
905582
거미
아니
928697
휘성 (Realslow)
일년이면
531840
프리스타일
Y (Please Tell Me Why)
848156
환희
가슴 아파도
845580
MC몽
천하무적 (Feat. 박장근 Of Supersta)
733379
쥬얼리
Super Star
884783
이정(J.Lee)
한숨만
891878
럼블피쉬
으라차차
623250
god
보통날
971439
리쌍
내가 웃는게 아니야 (Feat. ALI)
708213
조성모
눈물이 나요
891991
싸이 (PSY)
환희
926191
SG 워너비
내 마음의 보석상자
866706
김종국
그녀의 남자에게
732513
버즈
벌
842022
신혜성
같은 생각
864433
보아 (BoA)
Girls On Top
903404
장우혁
지지 않는 태양 (Feat. 한 에스더)
686651
izi


In [28]:
'''
멜론에서 시대별 차트
-> 맨 아래 top100
1번 -> 2020~2023
2번 -> 2010~2019
3번 -> 2000~2009
4번 -> 1990~1999
'''
start_year, max_year = 2000, 2004



cnt = 1
# max_year=2023 # 몇번째 페이지까지 넘길건지

data_dict = {"가수":[],
             "곡명":[],
             "가사":[],
             "장르":[],
             "발매일":[]
             }

# main문
driver = webdriver.Chrome()

for page_num in range(start_year, max_year+1):
    url = f"https://www.melon.com/chart/age/index.htm?chartType=YE&chartGenre=KPOP&chartDate={page_num}"

    # URL에서 HTML 가져오기
    driver.get(url)

    # BeautifulSoup을 사용하여 Selenium으로 열린 웹 페이지의 HTML 파싱
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 파싱된 HTML 출력
    # print(soup.prettify())
    # time.sleep(2)

    lst50 = soup.select("#lst50")
    lst100 = soup.select("#lst100")
    print(len(lst50), len(lst100))

    lst = lst50 + lst100

    for row in lst[:]:
        link_info = row.select("a")[1]
        print("="*50)
        # print(link_info.attrs["onclick"])

        num = link_info.attrs["onclick"].split("'")[-2]

        song_page = f"https://www.melon.com/song/detail.htm?songId={num}"
        print(num)
        print()
        driver.get(song_page)
        #######################################
        try : 
            artist = driver.find_elements(By.CLASS_NAME, 'artist')
            song_name = driver.find_elements(By.CLASS_NAME, 'song_name')
            lyric = driver.find_elements(By.CLASS_NAME, 'lyric')
            meta_data = driver.find_elements(By.CLASS_NAME, 'list')[0].text.split("\n")
            driver.implicitly_wait(60)
    
            lyrics = lyric[0].text
    
            print(artist[0].text) # 가수
            print(song_name[0].text) # 곡명
            # print(lyrics) # 가사
            # print(meta_data[5]) # 장르
            # print(meta_data[3]) # 발매일
            
            data_dict["가수"].append(artist[0].text)
            data_dict["곡명"].append(song_name[0].text)
            data_dict["가사"].append(lyrics)
            data_dict["장르"].append(meta_data[5])
            data_dict["발매일"].append(meta_data[3])
        except:
            print(song_page)
            
        # print(len(meta_data)) # 1

        # cnt = 1
        # for i in meta_data[0].text.split("\n"):
        #     print(f"{cnt} => {i}")
        #     cnt += 1

        
        #######################################

        #print()
        time.sleep(1)
        driver.back()  # 이전 창으로 복구

driver.quit()

df = pd.DataFrame.from_dict(data=data_dict)
print(df)

df.to_csv(f'melon_chart{start_year}to{max_year}.csv', sep=',', na_rep='NaN', encoding='utf-8')

50 50
106212
조성모
아시나요
1576735
조규만
다 줄거야 (Acoustic Ver.)
505165
DJ DOC
Run To You
10435
god
거짓말
91327
클론
초련(初戀) (Techno Mix) (Feat. 윤진)
607552
조성모
가시나무
87695
홍경민
흔들린 우정
86668
임창정
나의 연인(我戀)
85674
스카이
영원
38426
김현정
멍
4077
컨츄리 꼬꼬
오! 가니
27600
정일영
기도
46218
신승훈
전설속의 누군가처럼
3680
임재범
너를 위해
85108
박효신
바보
445159
이승환
그대가 그대를
9270
자우림
매직 카펫 라이드
2270749
조장혁
중독된 사랑
85707
박효신
해줄 수 없는 일
85900
박혜경
고백
75233
최재훈
비(悲)의 Rhapsody
88423
윤현석
Love
34093
김태영
오랜 방황의 끝
86166
플라이 투 더 스카이
Day By Day
1140592
소찬휘
Tears
67169
박상민
상실
88409
샵
잘됐어!!!
27833
박화요비
Lie (Radio Ver.)
3824
제이
어제처럼
3690
김경호
와인(迗人)
85257
이정현
바꿔
1002851
김현철
그대니까요 (Duet With 차은주)
85258
이정현
와
91329
원타임
One Love
610483
이현우
요즘 너는
86327
이소은
서방님
27236
쿨 (COOL)
맥주와 땅콩
1564057
이수영
I Believe
91325
god
애수(哀愁)
1606631
터보
Cyber Lover
33146
DJ DOC
머피의 법칙
91322
YB (윤도현 밴드)
너를 보내고
38541
김건모
부메랑
87310
백지영
Sad Salsa
37100
롤러 코스터
힘을 내요, 미스터 김
27638
박지윤
성인식
88794
서태지
울트라맨이야
82594
박기영
Blue Sky
27561
서문탁
사슬
87019
윤종신
Annie
27654
핑클 (Fin.K.L)
Now
91021
이정현
너
62185
엄정화
크로스


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=123.0.6312.86)
Stacktrace:
	GetHandleVerifier [0x00007FF7610F70C2+63090]
	(No symbol) [0x00007FF761062D12]
	(No symbol) [0x00007FF760EFEC65]
	(No symbol) [0x00007FF760EDCA7C]
	(No symbol) [0x00007FF760F6D687]
	(No symbol) [0x00007FF760F82AC1]
	(No symbol) [0x00007FF760F66D83]
	(No symbol) [0x00007FF760F383A8]
	(No symbol) [0x00007FF760F39441]
	GetHandleVerifier [0x00007FF7614F262D+4238301]
	GetHandleVerifier [0x00007FF76152F78D+4488509]
	GetHandleVerifier [0x00007FF761527A6F+4456479]
	GetHandleVerifier [0x00007FF7611D0606+953270]
	(No symbol) [0x00007FF76106E5DF]
	(No symbol) [0x00007FF7610692B4]
	(No symbol) [0x00007FF7610693EB]
	(No symbol) [0x00007FF761059C24]
	BaseThreadInitThunk [0x00007FFAD0FD257D+29]
	RtlUserThreadStart [0x00007FFAD328AA58+40]


In [29]:
df = pd.DataFrame.from_dict(data=data_dict)
print(df)

df.to_csv(f'melon_chart{start_year}to{max_year}.csv', sep=',', na_rep='NaN', encoding='utf-8')

                      가수                              곡명  \
0                    조성모                            아시나요   
1                    조규만           다 줄거야 (Acoustic Ver.)   
2                 DJ DOC                      Run To You   
3                    god                             거짓말   
4                     클론  초련(初戀) (Techno Mix) (Feat. 윤진)   
..                   ...                             ...   
226                  김현철                      Loving You   
227                Q.O.Q                            떠나가라   
228                 더 자두                         대화가 필요해   
229  체리필터 (cherryfilter)                           낭만고양이   
230                  이승환                         사랑하나요!?   

                                                    가사          장르         발매일  
0    아시나요 얼마나 사랑했는지\n그댈 보면 자꾸 눈물이 나서\n차마 그대 바라보지 못하...         발라드  2000.09.01  
1    그대 내게 다가오는 그 모습\n자꾸 다시 볼수 없을 것만 같아서\n감은 두눈 뜨지 ...         발라드  2000.01.01  
2    하나 투 원 투 쓰리 포\nBOUNCE with me B

In [31]:
'''
멜론에서 시대별 차트
-> 맨 아래 top100
1번 -> 2020~2023
2번 -> 2010~2019
3번 -> 2000~2009
4번 -> 1990~1999
'''
start_year, max_year = 1995, 1999



cnt = 1
# max_year=2023 # 몇번째 페이지까지 넘길건지

data_dict = {"가수":[],
             "곡명":[],
             "가사":[],
             "장르":[],
             "발매일":[]
             }

# main문
driver = webdriver.Chrome()

for page_num in range(start_year, max_year+1):
    url = f"https://www.melon.com/chart/age/index.htm?chartType=YE&chartGenre=KPOP&chartDate={page_num}"

    # URL에서 HTML 가져오기
    driver.get(url)

    # BeautifulSoup을 사용하여 Selenium으로 열린 웹 페이지의 HTML 파싱
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 파싱된 HTML 출력
    # print(soup.prettify())
    # time.sleep(2)

    lst50 = soup.select("#lst50")
    lst100 = soup.select("#lst100")
    print(len(lst50), len(lst100))

    lst = lst50 + lst100

    for row in lst[:]:
        link_info = row.select("a")[1]
        print("="*50)
        # print(link_info.attrs["onclick"])

        num = link_info.attrs["onclick"].split("'")[-2]

        song_page = f"https://www.melon.com/song/detail.htm?songId={num}"
        print(num)
        print()
        driver.get(song_page)
        #######################################
        try : 
            artist = driver.find_elements(By.CLASS_NAME, 'artist')
            song_name = driver.find_elements(By.CLASS_NAME, 'song_name')
            lyric = driver.find_elements(By.CLASS_NAME, 'lyric')
            meta_data = driver.find_elements(By.CLASS_NAME, 'list')[0].text.split("\n")
            driver.implicitly_wait(60)
    
            lyrics = lyric[0].text
    
            print(artist[0].text) # 가수
            print(song_name[0].text) # 곡명
            # print(lyrics) # 가사
            # print(meta_data[5]) # 장르
            # print(meta_data[3]) # 발매일
            
            data_dict["가수"].append(artist[0].text)
            data_dict["곡명"].append(song_name[0].text)
            data_dict["가사"].append(lyrics)
            data_dict["장르"].append(meta_data[5])
            data_dict["발매일"].append(meta_data[3])
        except:
            print(song_page)
            
        # print(len(meta_data)) # 1

        # cnt = 1
        # for i in meta_data[0].text.split("\n"):
        #     print(f"{cnt} => {i}")
        #     cnt += 1

        
        #######################################

        #print()
        time.sleep(1)
        driver.back()  # 이전 창으로 복구

driver.quit()

df = pd.DataFrame.from_dict(data=data_dict)
print(df)

df.to_csv(f'melon_chart{start_year}to{max_year}.csv', sep=',', na_rep='NaN', encoding='utf-8')

50 50
62462
김건모
잘못된 만남
1622501
녹색지대
사랑을 할꺼야
66261
솔리드
이 밤의 끝을 잡고
62098
015B
슬픈 인연
66052
K2 김성면
슬프도록 아름다운 ...
61240
장혜진
내게로
1839641
이소라
난 행복해
66311
노이즈
상상속의 너
63313
윤종신
부디
63343
한동준
사랑의 서약
66240
룰라
날개 잃은 천사
1698660
권진원
살다보면
421781
김광석
두바퀴로 가는 자동차
66156
피노키오
시간이 흐른뒤에
1620554
김혜림
날 위한 이별
33146
DJ DOC
머피의 법칙
58506
최재훈
널 보낸 후에...
607938
신효범
너의 곁에 있고 싶어
66366
R.ef
고요속의 외침
104067
노영심
그리움만 쌓이네
63353
https://www.melon.com/song/detail.htm?songId=63353
91491
성진우
포기하지 마
65305
박미경
이브의 경고
32854
더 블루
친구를 위해
63699
이승환
천일동안
91487
박진영
날 떠나지마
62922
조규찬
아담과 이브는 사과를 깨물었다
60807
박미경
이유같지 않은 이유
58937
신승훈
그 후로 오랫동안
71914
박정운
기억에 남는건 너의 젖은 눈동자
64242
박진영
청혼가
66411
베이시스
내가 날 버린 이유
66447
아낌없이 주는 나무
유년시절의 기행
1675397
듀스
굴레를 벗어나 (Mo Funk Ver.)
104941
김현철
끝난 건가요
61776
여행스케치
산다는 건 다 그런게 아니겠니
91124
이은미
어떤 그리움
66475
지니
뭐야 이건
991976
김원준
세상은 나에게
66507
https://www.melon.com/song/detail.htm?songId=66507
3793378
뱅크
가질수 없는 너
66397
모자이크
왕자와 병사들
1839375
DJ DOC
수퍼맨의 비애 (Feat. Proud Marry)
170430
최민수 (Flyger)
의미없는 시간
38931
강수지
흔들리

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=123.0.6312.86)
Stacktrace:
	GetHandleVerifier [0x00007FF7610F70C2+63090]
	(No symbol) [0x00007FF761062D12]
	(No symbol) [0x00007FF760EFEC65]
	(No symbol) [0x00007FF760EEC31B]
	(No symbol) [0x00007FF760EEBEC0]
	(No symbol) [0x00007FF760F00EC1]
	(No symbol) [0x00007FF760F82AE9]
	(No symbol) [0x00007FF760F66D83]
	(No symbol) [0x00007FF760F383A8]
	(No symbol) [0x00007FF760F39441]
	GetHandleVerifier [0x00007FF7614F262D+4238301]
	GetHandleVerifier [0x00007FF76152F78D+4488509]
	GetHandleVerifier [0x00007FF761527A6F+4456479]
	GetHandleVerifier [0x00007FF7611D0606+953270]
	(No symbol) [0x00007FF76106E5DF]
	(No symbol) [0x00007FF7610692B4]
	(No symbol) [0x00007FF7610693EB]
	(No symbol) [0x00007FF761059C24]
	BaseThreadInitThunk [0x00007FFAD0FD257D+29]
	RtlUserThreadStart [0x00007FFAD328AA58+40]


In [33]:
df = pd.DataFrame.from_dict(data=data_dict)
print(df)

df.to_csv(f'melon_chart{start_year}to{max_year}.csv', sep=',', na_rep='NaN', encoding='utf-8')

           가수              곡명  \
0         김건모          잘못된 만남   
1        녹색지대         사랑을 할꺼야   
2         솔리드      이 밤의 끝을 잡고   
3        015B           슬픈 인연   
4      K2 김성면   슬프도록 아름다운 ...   
..        ...             ...   
443  Monocrom  니가 진짜로 원하는게 뭐야   
444       김민종              순수   
445       이문세          애수(哀愁)   
446       이승환          세가지 소원   
447       코요태         순정 (純情)   

                                                    가사        장르         발매일  
0    난 너를 믿었던 만큼\n난 내 친구도 믿었기에\n난 아무런 부담없이 널\n내 친구에...        댄스  1995.01.24  
1    이젠 나도 널 잊겠어\n너무 힘이 들잖아\n원하는 대로 해줄 순 있지만\n난 더 이...       발라드  1994.01.01  
2    다신 널 볼 수 없겠지\n나의 입술이 너의 하얀\n어깨를 감싸 안으며\n그렇게 우린...  R&B/Soul  1995.03.01  
3    멀어져 가는 저 뒷모습을\n바라보면서\n난 아직도 이 순간을 이별이라\n하지 않겠네...       발라드  1994.11.20  
4    그리움의 끝엔 언제나\n눈물 속에 항상 네가 있는 것은\n돌아갈 수 없는 아픔인듯 ...      록/메탈  1995.01.15  
..                                                 ...       ...         ...  
443  사는 대로 사니 가는 대로 사니\n그냥 되는 대로 사니\n사사 사는 대로 사니\n